In [83]:
import numpy as np
import random as rnd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from IPython.display import display, clear_output
# import ipywidgets.interact class, this class will represent a slide bar.
from ipywidgets import interact
from matplotlib.colors import LinearSegmentedColormap
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

from multiprocessing import Pool, Lock, cpu_count,Manager
manager = Manager()
results = manager.list()
cpu = cpu_count()
print(cpu)

8


From:

[Landis, Geoffrey A. "The Fermi paradox: an approach based on percolation theory." Journal of the british interplanetary society 51.5 (1998): 163-166](http://www.geoffreylandis.com/percolation.htp)


Space is a 3D grid. each cell of the grid ( a sector) is linked to its 6 immediate Von Numan neighbours. Each sector can be in one of three states:
 
1: Uncolonized
 
2: Colonised by colonising colony
 
3: Colonised by a non colonising colony

Algorithm:
 
Each colonising colony searches its 6 immediate sectors for an uncolonized sector. If one is found then it is colonised with a colonising colony with a probability of p and a a non colonising colony with a probability 1-p



Sector class

This is a unit of 3d space

It contains a state and a link to its 6 imediate neighbours

Attributes

            colonized -> int

            x_pos ->int

            y_pos ->int

            z_pos ->int

            neighbours-> list of Space

            capasity -> boolean # if there are no empty neighbours

Methods

            add_neighbour

            colonize

            get_pos -> x,y,z


In [84]:
class Sector:
    colinising = 0.5
    def __init__(self,x_pos,y_pos,z_pos):
        self.id = str(x_pos)+":"+str(y_pos)+":"+str(z_pos)
        # give slight random stutter in xyz to produce a more 'realistic' plot
        self.x_pos = x_pos + 0.5-rnd.random()
        self.y_pos = y_pos + 0.5-rnd.random()
        self.z_pos = z_pos + 0.5-rnd.random()
        self.neighbours =[]
        self.colonized=0
        self.edge = True

    def add_neighbour(self,sector):
        self.neighbours.append(sector)

    def colonize(self):
        if len(self.neighbours)>0:
            sector = rnd.choice(self.neighbours)
            while sector.colonized>0 and len(self.neighbours)>0:
                self.neighbours.remove(sector)
                if len(self.neighbours)>0:
                    sector = rnd.choice(self.neighbours)
                else:
                    return False
            if rnd.random()<Sector.colinising:
                sector.colonized=1
            else:
                sector.colonized=2
            self.neighbours.remove(sector)
            return sector
        else:
             return False

    def get_pos(self):
        c_type= "un colonized"
        if self.colonized == 1:
            c_type = "spacefaring"
        if self.colonized == 2:
            c_type = "non-spacefaring"    
        return self.x_pos,self.y_pos,self.z_pos,c_type


Space class


Attributes

            size -> int
            
            colonies -> list of Space

            space_faring -> list of Space

            space_chart ->list of np lists (x,y,z,c)


Methods

            init_space

            iterate

            plot


In [85]:

class Space:
    def __init__(self,size) -> None:
        def bounds(v):
            if v>=size: 
                return False
            if v<0:
                return False
            return True
            
        self.size = size
        self.edge = False
        # crate sectors and link in network
        temp_space=[]
        for x in range(size):
            temp_space.append([])
            for y in range(size):
                temp_space[x].append([])
                for z in range(size):
                    temp_space[x][y].append(Sector(x,y,z))
        for x in range(1,size-1):
            for y in range(1,size-1):
                for z in range(1,size-1):
                    temp_space[x][y][z].edge = False
                    temp_space[x][y][z].add_neighbour(temp_space[x-1][y][z]) 
                    temp_space[x][y][z].add_neighbour(temp_space[x+1][y][z]) 
                    temp_space[x][y][z].add_neighbour(temp_space[x][y-1][z]) 
                    temp_space[x][y][z].add_neighbour(temp_space[x][y+1][z]) 
                    temp_space[x][y][z].add_neighbour(temp_space[x][y][z-1]) 
                    temp_space[x][y][z].add_neighbour(temp_space[x][y][z+1]) 
        
        home_world = temp_space[x//2][y//2][z//2] 
        home_world.colonized=1
        self.colonies = [home_world]
        self.space_faring = [home_world]
        x,y,z,c = home_world.get_pos()
        self.star_map = []
        self.star_map.append([x,y,z,c])

    def iterate(self):
        
        new_worlds = []
        settled = []

        for c in self.space_faring:
            new_world = c.colonize()
            if new_world != False:
                self.colonies.append(new_world)
                x,y,z,c_type = new_world.get_pos()
                self.star_map.append([x,y,z,c_type])
                if new_world.colonized==1:
                    new_worlds.append(new_world)
                if new_world.edge:
                    self.edge= True

            if len(c.neighbours)==0:
                settled.append(c)


        for s in settled:
            self.space_faring.remove(s)
        for n in new_worlds:
            self.space_faring.append(n)
                    
    
        return len(self.space_faring)==0, self.edge


                    
    

In [86]:
# seed = rnd.randrange(1000000)
# seed=145138
# rnd.seed(seed)
def experiment(p):
    Sector.colinising=p
    size =50
    asize=size-2
    for r in range(10):
        state = "expanding"
        exp = Space(size)
        for i in range(200):
            flag, edge = exp.iterate()
            if flag:
                state = "dormant"
                break
        if edge:
            state = "edge"
        results.append([p,r,len(exp.star_map)/(size*size*size),state])


In [ ]:



p = Pool(cpu)
result = p.map(experiment, [i/100 for i in range(60)])

In [ ]:
print("done")

In [ ]:
df = pd.DataFrame(list(results), columns=['probability_of_spacefaring','rep','fraction_of_space_colonised','state'])

In [ ]:
fig = px.scatter(df, x="probability_of_spacefaring", y="fraction_of_space_colonised", color="state",log_y=True)
fig.show()

In [ ]:
stats = df.groupby("probability_of_spacefaring").agg({"fraction_of_space_colonised":['mean','std']})

In [ ]:
stats.head()

In [ ]:
fig = go.Figure(data=go.Scatter(
        x=stats.index,
        y=stats["fraction_of_space_colonised"]["mean"],
        mode='markers',
        error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=stats["fraction_of_space_colonised"]["std"],
            visible=True)
    ))
#fig.update_yaxes(type="log")
fig.show()

In [ ]:
fig = go.Figure(data=go.Scatter(
        x=stats.index,
        y=stats["fraction_of_space_colonised"]["mean"],
        mode='markers'
    ))
fig.update_yaxes(type="log")
fig.show()